# PREPARATIVOS

In [1]:
import pandas as pd
import numpy as np
import itertools as it
import re
import os
from funciones import *

In [2]:
%load_ext line_profiler

In [3]:
FBL3N = leer_excel("FBL3N.xlsx")

Leyendo archivo Excel: FBL3N.xlsx
Archivo Excel leído.


In [4]:
FBL5N = leer_excel("FBL5N.xlsx")

Leyendo archivo Excel: FBL5N.xlsx
Archivo Excel leído.


In [5]:
KNA1 = leer_excel("KNA1.xlsx")

Leyendo archivo Excel: KNA1.xlsx
Archivo Excel leído.


In [6]:
#PROPUESTA = leer_excel("PROPUESTA.xlsx", None)

# PARTE 1

In [7]:
textos = FBL3N["Texto"]
FBL3N["RUT"] = textos.apply(detectar_rut)
FBL3N["N° Factura"] = textos.apply(detectar_n_factura)

RUT_FBL3N = FBL3N[pd.notna(FBL3N["RUT"])]
FACT_FBL3N = FBL3N[pd.isna(FBL3N["RUT"]) & pd.notna(FBL3N["N° Factura"])]

# PARTE 2

### Trabajar con calces de factura

In [8]:
FBL5N["Referencia"] = FBL5N["Referencia"].apply(lambda s: str(s))
FBL5N["Cuenta"] = FBL5N["Cuenta"][pd.notna(FBL5N["Cuenta"])].apply(lambda n: str(int(n)))

KNA1["Nº ident.fis.1"] = KNA1["Nº ident.fis.1"].apply(lambda s: str(s).lstrip("0"))
KNA1["Cliente"] = KNA1["Cliente"][pd.notna(KNA1["Cliente"])].apply(lambda s: str(int(s)))

In [9]:
# Calzar números de factura
filtro_fact = FBL5N["Referencia"].isin(FACT_FBL3N["N° Factura"])
FACT_FBL5N = FBL5N[filtro_fact]
NO_FACT_FBL5N = FBL5N[~filtro_fact]

In [10]:
# Calzar RUTs
ID_FBL3N = pd.merge(RUT_FBL3N, KNA1[["Nº ident.fis.1", "Cliente"]], left_on="RUT", right_on="Nº ident.fis.1")

filtro_id = NO_FACT_FBL5N["Cuenta"].isin(ID_FBL3N["Cliente"])
ID_FBL5N = NO_FACT_FBL5N[filtro_id]
NN_FBL5N = NO_FACT_FBL5N[~filtro_id]

In [11]:
try:
    os.mkdir("RESULTADOS/")
except:
    pass

In [12]:
MATCHES_FACT = pd.merge(FACT_FBL3N, FACT_FBL5N, left_on="N° Factura", right_on="Referencia")
escribir_excel(FACT_FBL5N, "RESULTADOS/Facturas.xlsx")

Escribiendo nuevo archivo Excel: RESULTADOS/Facturas.xlsx
Archivo Excel escrito.


# PARTE 3

### Trabajar con calces de ID

In [13]:
def match(ID_FBL3N, ID_FBL5N):
    def descartar(merge, match):
        return merge[(~merge["N° documento"].isin(match["N° documento"])) & (~merge["N° doc."].isin(match["N° doc."]))]
    
    sort = ID_FBL5N.sort_values(by="Demora", ascending="False")
    merge = pd.merge(ID_FBL3N, sort, left_on="Cliente", right_on="Cuenta", suffixes=(" FBL3N", " FBL5N"))
    
    # Match uno-a-uno
    match_1 = merge[-merge["Importe en moneda local"] == merge["Importe en ML"]]
    match_1 = match_1.drop_duplicates(subset="N° documento")
    
    merge = descartar(merge, match_1)
    
    """
    # Match uno-a-todos
    match_t = pd.concat(
        [merge[
            -merge[(merge["N° documento"] == doc) & (merge["Cliente"] == ID)]["Importe en moneda local"]
            == merge[(merge["N° documento"] == doc) & (merge["Cliente"] == ID)]["Importe en ML"].sum()
            ]
        for doc, ID in zip(merge["N° documento"], merge["Cliente"])]
    )
    
    merge = descartar(merge, match_t)
    """
    
    return match_1

In [14]:
MATCHES_ID_1_1 = match(ID_FBL3N, ID_FBL5N)
escribir_excel(match(ID_FBL3N, ID_FBL5N), "RESULTADOS/RUT y Monto 1-1.xlsx")

Escribiendo nuevo archivo Excel: RESULTADOS/RUT y Monto 1-1.xlsx
Archivo Excel escrito.


## Matches uno-a-uno

In [35]:
def agrupar(ids3, doc3, mon3, ids5, doc5, mon5):
    
    """ PARTE 1: AGRUPAR DATOS """
    
    ids_uni = np.intersect1d(ids3, ids5)
    
    # Ordenar IDs, documentos y montos en base a las IDs
    ind3_ord = np.argsort(ids3)
    ids3_ord = ids3[ind3_ord]
    doc3_ord = doc3[ind3_ord]
    mon3_ord = mon3[ind3_ord]
    
    ind5_ord = np.argsort(ids5)
    ids5_ord = ids5[ind5_ord]
    doc5_ord = doc5[ind5_ord]
    mon5_ord = mon5[ind5_ord]
    
    # Registrar los cambios de grupos, es decir, cuándo un elemento ya no es igual al siguiente
    # Para eso, comparar el arreglo de IDs con una copia desplazada de sí mismo
    lim3 = np.where(np.append([""], ids3_ord) != np.append(ids3_ord, [""]))[0]
    lim5 = np.where(np.append([""], ids5_ord) != np.append(ids5_ord, [""]))[0]
    
    # Obtener los grupos
    n_ids = len(ids_uni)
    lista = [
        (
            doc3_ord[lim3[i] : lim3[i+1]],
            mon3_ord[lim3[i] : lim3[i+1]],
            doc5_ord[lim5[i] : lim5[i+1]],
            mon5_ord[lim5[i] : lim5[i+1]]
        ) for i in range(n_ids)
    ]
    
    doc3_grp, mon3_grp, doc5_grp, mon5_grp = zip(*lista)
    
    """ PARTE 2: FILTRAR DATOS """
    # Por hacer
    
    return pd.DataFrame({"ID": ids_uni, "Docs FBL3N": doc3_grp, "Montos FBL3N": mon3_grp, "Docs FBL5N": doc5_grp, "Montos FBL5N": mon5_grp})

In [36]:
ids3 = ID_FBL3N["Cliente"].values
doc3 = ID_FBL3N["N° documento"].values
mon3 = ID_FBL3N["Importe en moneda local"].values

ids5 = ID_FBL5N["Cuenta"].values
doc5 = ID_FBL5N["N° doc."].values
mon5 = ID_FBL5N["Importe en ML"].values

agrupar(ids3, doc3, mon3, ids5, doc5, mon5)

,ID,Docs FBL3N,Montos FBL3N,Docs FBL5N,Montos FBL5N
0,1000988,[100401286.0],[-66685],[2500335936.0],[194625]
1,1001034,[100445348.0],[-8252],"[1400042005.0, 2500378705.0, 2500360333.0, 250...","[-1, 61724, 76302, 100738, 117357, 97902, -1, ..."
2,1001047,[100411847.0],[-1287870],"[2500377562.0, 2500370978.0, 2500372265.0, 250...","[120254, 42322, 3639, 11662, 5831, 424650, 361..."
3,1001100,"[100338256.0, 100338142.0]","[-1, -1]",[2500330923.0],[211233]
4,1001121,[100466705.0],[-22063],"[2500355252.0, 2500296204.0, 2500335870.0, 250...","[93505, 118785, 65938, 6719, 40772, 68580, -1]"
...,...,...,...,...,...
647,3322207,[100407508.0],[-125408],"[100227114.0, 100385843.0]","[-5676, -1178]"
648,3326293,[100268983.0],[-19016],"[2500379006.0, 1400180457.0, 2500378692.0]","[498397, -667125, 168728]"
649,3326496,[100413824.0],[-198233],"[100360366.0, 2500275607.0, 2500275837.0, 2500...","[-214408, 72809, 69601, 115014, 60720, 122472,..."
650,3343520,[100166130.0],[-224650],"[1400168046.0, 2500352281.0]","[-95989, 119336]"


In [17]:
%lprun -f agrupar agrupar(ids3, doc3, mon3, ids5, doc5, mon5)g

In [20]:
%timeit agrupar(ids3, doc3, mon3, ids5, doc5, mon5)

13.6 ms ± 102 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [18]:
MATCHES_ID = pd.merge(ID_FBL3N, ID_FBL5N, left_on="Cliente", right_on="Cuenta", suffixes=(" FBL3N", " FBL5N"))

In [19]:
pd.concat([MATCHES_ID[(MATCHES_ID["Nº documento"] == doc) & (MATCHES_ID["Cliente"] == ID)] for doc, ID in zip(MATCHES_ID["Nº documento"], MATCHES_ID["Cliente"])])

KeyError: 'Nº documento'

In [ ]:
ID_FBL3N[
    pd.concat(
        [-ID_FBL3N[(ID_FBL3N["N° documento"] == doc) & (ID_FBL3N["Cliente"] == ID)]["Importe en moneda local"]
         == ID_FBL5N[ID_FBL5N["Cuenta"] == ID]["Importe en ML"].sum()
         for doc, ID in zip(ID_FBL3N["N° documento"], ID_FBL3N["Cliente"])]
    ).reset_index().drop_duplicates(subset="index")["Importe en moneda local"]
].to_excel("RESULTADOS/RUT y Monto 1-T.xlsx")

# PARTE 4

### Determinar qué datos encontré yo que Roberto no, y viceversa

1. RUT y Monto

In [ ]:
RUT_Y_MONTO = leer_excel("PROPUESTA/RUT y Monto.xlsx")

In [ ]:
outer_rym = pd.merge(MATCHES_ID_1_1, RUT_Y_MONTO, on="N° documento", how="outer", indicator=True)

In [ ]:
try:
    os.mkdir("COMPARACIONES/")
except:
    pass

In [ ]:
escribir_excel(outer_rym[outer_rym["_merge"] == "left_only"], "COMPARACIONES/RUT y Monto - LEFT.xlsx")

In [ ]:
escribir_excel(outer_rym[outer_rym["_merge"] == "right_only"], "COMPARACIONES/RUT y Monto - RIGHT.xlsx")

2. Solo RUT

In [ ]:
SOLO_RUT = leer_excel("002_SoloRut.xlsx")

In [ ]:
outer_sr = pd.merge(MATCHES_ID, SOLO_RUT, on="Nº documento", how="outer", indicator=True)

In [ ]:
escribir_excel(outer_sr[outer_sr["_merge"] == "left_only"], "SOLO_RUT_LEFT.xlsx")

In [ ]:
escribir_excel(outer_sr[outer_sr["_merge"] == "right_only"], "SOLO_RUT_RIGHT.xlsx")